<a href="https://colab.research.google.com/github/pgallino/Organizacion-de-datos-TP1-1C2023/blob/main/pandas51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


51) Si la experiencia de un usuario es la cantidad de logs en los que participó, queremos saber que tanto nos sirve para predecir el futuro vandalismo: ¿Cuál es la probabilidad de que un usuario sea bloqueado según experiencias: <10, 10-40, 40-100, >100? Tener en cuenta que esta experiencia debe ser PREVIA al bloqueo del usuario. (⭐⭐⭐)

HAY DOS RESOLUCIONES. UNA ES CORRIGIENDO LO MARCADO EN LA PREENTREGA Y OTRA ES IMPLEMENTANDO LA RESOLUCIÓN SUGERIDA EN LA PREENTREGA POR EL CORRECTOR. LA RESOLUCIÓN "CORRECTA" ES LA SEGUNDA. EL COMIENZO ES IGUAL PARA AMBAS.

Para analizar la cantidad de logs en los que participó cada usuario, utilizaré el timestamp, contributor_username, action y title. Si son usuarios, tienen contributor_username. Cada username, es único, por lo que lo usaré para identificarlos.

In [ ]:
df_logs = pd.read_csv('/content/drive/MyDrive/Wikipedia dataset 2c2021/logs.csv', usecols = [1,2,7,8])
df_logs.head(20)

Para poder contar los logs hasta el primer 'block', tengo que identificar el timestamp en la que fue bloqueado cada usuario.

El username del usuario bloqueado no es 'contributor_username', sino que figura en 'title' como Usuario:'username'.

In [ ]:
df_blocks = df_logs.loc[(df_logs.action == 'block') & ~(df_logs.title.isnull())]
df_blocks = df_blocks[['timestamp','title']]
df_blocks["title"] = df_blocks.title.map(lambda x : x.split(":")[1]) #limpio el "usuario:"
df_blocks = df_blocks.rename(columns={'title':'username', 'timestamp': 'tiempo_bloqueo'})

In [ ]:
df_blocks

,tiempo_bloqueo,username
5,2004-12-23T08:28:32Z,212.55.30.157
287,2004-12-27T17:54:50Z,201.241.228.15
288,2004-12-27T17:57:12Z,201.255.50.32
290,2004-12-27T17:59:56Z,84.122.0.158
313,2004-12-28T01:38:59Z,81.192.12.93
...,...,...
12188865,2021-09-03T02:05:46Z,186.141.200.121
12188886,2021-09-03T02:24:41Z,Galaxi066
12188909,2021-09-03T02:52:42Z,Xx.tkpro.xx
12188924,2021-09-03T03:00:14Z,181.167.104.152


Como pueden ser bloqueados más de una vez, me quedo con la primera vez.

In [ ]:
df_blocks = df_blocks.groupby('username', as_index = False).agg({"tiempo_bloqueo":"min"})

Una vez obtenido el tiempo de bloqueo de cada usuario, lo uno al dataframe original para poder contar los logs, comparando si se realizó antes o después del mismo.

In [ ]:
df_logs = df_logs.rename(columns={'contributor_username':'username'})
df_total = pd.merge(df_logs , df_blocks, how = 'left')

Añado una columna con la info de si fue bloqueado o no alguna vez el usuario que realizo el log.

In [ ]:
df_total["bloqueado"] = ~df_total.tiempo_bloqueo.isnull()

In [ ]:
df_total.loc[df_total['bloqueado']]

,timestamp,username,action,title,tiempo_bloqueo,bloqueado
0,2004-12-23T07:10:09Z,Sabbut,delete,Verificacionismo,2010-12-15T13:07:57Z,True
1,2004-12-23T07:14:09Z,Sabbut,delete,Rapunzel,2010-12-15T13:07:57Z,True
2,2004-12-23T07:26:10Z,Sabbut,delete,Macromedia Flash,2010-12-15T13:07:57Z,True
3,2004-12-23T07:55:09Z,Sanbec,delete,Archivo:Gnu head.png,2005-07-22T11:15:19Z,True
4,2004-12-23T08:02:22Z,Sanbec,delete,Archivo:Gnu-head-sm.jpg,2005-07-22T11:15:19Z,True
...,...,...,...,...,...,...
12188143,2021-09-02T19:24:49Z,David Sacristán,create,República de Cuevaland de los 3 centímetros cu...,2021-09-03T00:27:31Z,True
12188527,2021-09-02T23:03:42Z,Jason Daily,create,"La Película ""Noche De Odio""",2021-09-03T00:18:49Z,True
12188542,2021-09-02T23:11:30Z,Pervertidosinremedio,create,Usuario:Pervertidosinremedio,2021-09-02T23:13:25Z,True
12188548,2021-09-02T23:14:35Z,JohnWayne454545,create,Usuario:JohnWayne454545,2021-09-03T00:39:46Z,True


HASTA ACA COMPARTEN LAS DOS RESOLUCIONES

________________________________________________________________________________

PRIMERA RESOLUCION - SEPARANDO EN DOS DATAFRAMES

Separo el dataframe entre los usuarios bloqueados y los que no, lo cual me servirá para calcular la probabilidad con LAPLACE. Si el espacio es equiprobable, hago casos favorables sobre casos totales.

In [ ]:
df_bloqueados = df_total[[x for x in df_total['bloqueado']]]
df_no_bloqueados = df_total[[not x for x in df_total['bloqueado']]]

Filtro todos los logs de los usuarios bloqueados antes de que sean bloqueados.

In [ ]:
df_bloqueados = df_bloqueados[[x.total_seconds() > 0 for x in (df_bloqueados['tiempo_bloqueo'] - df_bloqueados['timestamp'])]]

In [ ]:
df_total.loc[df_total['action'] == 'block'] = df_total.loc[df_total["tiempo_bloqueo"] > df_total["tiempo_registro"]]

En este punto, ya tengo dos dataframes con todos los logs realizados por usuarios bloqueados y no bloqueados. De ellos, solo necesito el 'username' para agruparlos.

In [ ]:
df_bloqueados = df_bloqueados[['username']]
df_no_bloqueados = df_no_bloqueados[['username']]

In [ ]:
df_bloqueados

,username
0,Sabbut
1,Sabbut
2,Sabbut
3,Sanbec
4,Sanbec
...,...
12188143,David Sacristán
12188527,Jason Daily
12188542,Pervertidosinremedio
12188548,JohnWayne454545


In [ ]:
df_no_bloqueados

,username
14,Ascánder
16,Ascánder
17,Ascánder
18,Ascánder
19,Ascánder
...,...
12188940,Yosoytunotuyosoytuaaa
12188942,Leonpolanco
12188943,Desiretjs
12188944,Correogsk


Agrego una columna 'cant_logs' con 1 para cuantificar la cantidad de logs de cada usuario.

In [ ]:
df_bloqueados['cant_logs'] = 1
df_no_bloqueados['cant_logs'] = 1

<ipython-input-21-fc2b71113524>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_bloqueados['cant_logs'] = 1


Cuento los logs por usuario.

In [ ]:
df_bloqueados = df_bloqueados.groupby(['username']).count()
df_no_bloqueados = df_no_bloqueados.groupby(['username']).count()

Puedo ver la cantidad de logs realizados por cada usuario y ordenarlos.

In [ ]:
df_bloqueados.sort_values(['cant_logs'])

,cant_logs
username,
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1
Miunicornio.com,1
Miumpyli,1
Mitzukiuchiha14,1
Mitus82,1
...,...
Yrithinnd,21779
Hprmedina,26971
Dodo,28788


In [ ]:
df_no_bloqueados.sort_values(['cant_logs'])

,cant_logs
username,
! ! ! Histoire hebdomadaire de la patrie ! ! !,1
Michelleasq,1
Michellearogers,1
Michellearmy,1
Michellearlette,1
...,...
Marcelo,90044
Humberto,101765
Antur,112565


Defino una función para agilizar el cálculo de la probabilidad. Recibe el mínimo y el máximo de experiencias.
Para calcularla, obtengo la cantidad de bloqueados con ese rango de experiencias y la divido sobre la cantidad de usuarios totales con ese rango de experiencias.

In [ ]:
def prob_ser_bloqueado_por_cant_logs(min, max):
  cant_bloqueados = len((df_bloqueados[[ min <= x < max for x in df_bloqueados['cant_logs']]]).index)
  cant_no_bloqueados = len((df_no_bloqueados[[ min <= x < max for x in df_no_bloqueados['cant_logs']]]).index)
  prom = cant_bloqueados/(cant_bloqueados + cant_no_bloqueados)
  return prom

Puedo ver la probabilidad de ser bloqueado para un usuario con menos de 10 experiencias = 0.015 aproximadamente.

In [ ]:
prob_ser_bloqueado_por_cant_logs(0,10)

0.015154709858636196

Puedo ver la probabilidad de ser bloqueado para un usuario con un rango de [10;40) experiencias = 0.1857 aproximadamente.

In [ ]:
prob_ser_bloqueado_por_cant_logs(10,40)

0.18572614107883817

Puedo ver la probabilidad de ser bloqueado para un usuario con un rango de [40,100) experiencias = 0.1730 aproximadamente.

In [ ]:
prob_ser_bloqueado_por_cant_logs(40,100)

0.1730225988700565

Puedo ver la probabilidad de ser bloqueado para un usuario con 100 o más experiencias = 0.1754 aproximadamente.

In [ ]:
prob_ser_bloqueado_por_cant_logs(100,1000000000000)

0.1753972925250147

________________________________________________________________________________

SEGUNDA RESOLUCIÓN - SIN SEPARAR EN DOS DATAFRAMES

In [ ]:
df_total = df_total.loc[(~df_total['bloqueado']) | (df_total['bloqueado'] & (df_total['tiempo_bloqueo'] > df_total['timestamp']))]

In [ ]:
df_total

,timestamp,username,action,title,tiempo_bloqueo,bloqueado
0,2004-12-23T07:10:09Z,Sabbut,delete,Verificacionismo,2010-12-15T13:07:57Z,True
1,2004-12-23T07:14:09Z,Sabbut,delete,Rapunzel,2010-12-15T13:07:57Z,True
2,2004-12-23T07:26:10Z,Sabbut,delete,Macromedia Flash,2010-12-15T13:07:57Z,True
3,2004-12-23T07:55:09Z,Sanbec,delete,Archivo:Gnu head.png,2005-07-22T11:15:19Z,True
4,2004-12-23T08:02:22Z,Sanbec,delete,Archivo:Gnu-head-sm.jpg,2005-07-22T11:15:19Z,True
...,...,...,...,...,...,...
12188940,2021-09-03T03:08:58Z,Yosoytunotuyosoytuaaa,byemail,Usuario:Magikyty23809pynpynfyte,NaN,False
12188942,2021-09-03T03:09:48Z,Leonpolanco,create,Usuario discusión:2806:2F0:3140:BE0:C09B:D1C4:...,NaN,False
12188943,2021-09-03T03:09:49Z,Desiretjs,create,Usuario:Desiretjs,NaN,False
12188944,2021-09-03T03:10:33Z,Correogsk,create,Contenedores,NaN,False


In [ ]:
df_total = df_total.groupby(['username'], as_index = False).agg({'action':'count','bloqueado':'first'})

In [ ]:
df_total.sort_values(['action'])

,username,action,bloqueado
0,! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,1,True
4092114,Michi-kun,1,False
4092113,Michi-fcn,1,False
4092112,Michi-chan,1,False
4092111,Michi- Joseph,1,False
...,...,...,...
2476345,Humberto,101765,False
519056,Antur,112565,False
2554497,InternetArchiveBot,163770,True
5531207,Taichi,193345,False


In [ ]:
def prob_ser_bloqueado_por_cant_logs(min, max):
  cant_block = (df_total.loc[(df_total.bloqueado) & ( (min <= df_total.action) & (df_total.action < max)), 'action']).count()
  cant_total = (df_total.loc[ (min <= df_total.action) & (df_total.action < max), 'action']).count()
  return cant_block/cant_total

In [ ]:
prob_ser_bloqueado_por_cant_logs(0,10)

0.015154709858636196

In [ ]:
prob_ser_bloqueado_por_cant_logs(10,40)

0.18572614107883817

In [ ]:
prob_ser_bloqueado_por_cant_logs(40,100)

0.1730225988700565

In [ ]:
prob_ser_bloqueado_por_cant_logs(100,1000000000000)

0.1753972925250147